In [1]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# --- setup ---
pd.set_option('max_columns', 50)

In [2]:
vaccination = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
vaccination

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13921,Zimbabwe,ZWE,2021-04-20,325007.0,288229.0,36778.0,8016.0,10950.0,2.19,1.94,0.25,737.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1386047507...
13922,Zimbabwe,ZWE,2021-04-21,332996.0,295631.0,37365.0,7989.0,10124.0,2.24,1.99,0.25,681.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1386047507...
13923,Zimbabwe,ZWE,2021-04-22,351799.0,308071.0,43728.0,18803.0,9417.0,2.37,2.07,0.29,634.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1386047507...
13924,Zimbabwe,ZWE,2021-04-23,365985.0,319732.0,46253.0,14186.0,8755.0,2.46,2.15,0.31,589.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1386047507...


In [3]:
europe_country_list =list([
    'Austria','Belgium','Bulgaria','Czechia','Denmark','France','Germany','Greece',
    'Italy','Luxembourg','Lithuania','Norway','Poland','Portugal','Romania','Slovakia','Slovenia',
    'Spain', 'United Kingdom', 'Russia', 'Serbia', 'Ukraine'])

vaccination = vaccination[vaccination['country'].isin(europe_country_list)]
vaccination.shape

(2536, 15)

In [4]:
vaccination = vaccination.drop(['iso_code', 'source_name', 'source_website'], axis = 1).reset_index()
vaccination

,index,country,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines
0,687,Austria,2020-12-27,732.0,732.0,NaN,NaN,NaN,0.01,0.01,NaN,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
1,688,Austria,2020-12-28,2031.0,2031.0,NaN,1299.0,1299.0,0.02,0.02,NaN,144.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
2,689,Austria,2020-12-29,3651.0,3651.0,NaN,1620.0,1460.0,0.04,0.04,NaN,162.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
3,690,Austria,2020-12-30,5004.0,5004.0,NaN,1353.0,1424.0,0.06,0.06,NaN,158.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
4,691,Austria,2020-12-31,5012.0,5012.0,NaN,8.0,1070.0,0.06,0.06,NaN,119.0,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,13415,United Kingdom,2021-04-20,43915559.0,33139742.0,10775817.0,457649.0,488411.0,64.69,48.82,15.87,7195.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
2532,13416,United Kingdom,2021-04-21,44450252.0,33257651.0,11192601.0,534693.0,498850.0,65.48,48.99,16.49,7348.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
2533,13417,United Kingdom,2021-04-22,45012308.0,33388637.0,11623671.0,562056.0,500934.0,66.31,49.18,17.12,7379.0,"Oxford/AstraZeneca, Pfizer/BioNTech"
2534,13418,United Kingdom,2021-04-23,45613109.0,33524423.0,12088686.0,600801.0,496017.0,67.19,49.38,17.81,7307.0,"Oxford/AstraZeneca, Pfizer/BioNTech"


In [5]:
vcc_all = vaccination.drop(['vaccines'], axis = 1)
vcc_all = vcc_all.groupby(['date']).sum()
vcc_all.reset_index(inplace=True)
vcc_all

,date,index,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,2020-12-15,10619,28500.0,28500.0,0.0,0.0,0.0,0.02,0.02,0.00,0.0
1,2020-12-16,10620,0.0,0.0,0.0,0.0,3357.0,0.00,0.00,0.00,23.0
2,2020-12-17,10621,0.0,0.0,0.0,0.0,3357.0,0.00,0.00,0.00,23.0
3,2020-12-18,10622,0.0,0.0,0.0,0.0,3357.0,0.00,0.00,0.00,23.0
4,2020-12-19,10623,0.0,0.0,0.0,0.0,3357.0,0.00,0.00,0.00,23.0
...,...,...,...,...,...,...,...,...,...,...,...
127,2021-04-21,172482,171442899.0,123707045.0,47735832.0,3591202.0,2931944.0,592.91,423.13,169.74,101462.0
128,2021-04-22,172504,175231196.0,126161067.0,49070422.0,3788297.0,2956057.0,606.54,432.45,174.09,100672.0
129,2021-04-23,155214,177011361.0,127054983.0,49957822.0,3450063.0,2920004.0,562.40,397.72,164.68,91834.0
130,2021-04-24,143012,154773833.0,110089395.0,44690171.0,2620821.0,2695757.0,511.45,359.07,152.40,86619.0


In [6]:

vaccination.fillna(method='ffill', inplace=True)
countries = vaccination.groupby('country')['total_vaccinations'].max().sort_values(ascending=False)[:10].index.tolist()

line_plots = []
for c in countries:
    vacc_data = vaccination[vaccination.country == c]
    line_plots.append(
        go.Scatter(
            name = c,
            x = vacc_data.date,
            mode='lines',
            y=vacc_data['total_vaccinations'],
        )
    )
    
fig = go.Figure(line_plots)
fig.update_layout(
    title ="Rate of vaccinations for the top-10 vaccinated countries",
    yaxis_title="Count",
    hovermode='x',
    legend_orientation = 'h',

)


fig.show()

In [7]:

fig = px.line(vaccination, x="date", y="people_vaccinated_per_hundred", color='country', 
              title="date vs people_vaccinated_per_hundred")
fig.show()

In [8]:
# Grouping data by vaccines
vaccines = vaccination.groupby(['country', 'vaccines']).count().reset_index()[['country', 'vaccines']]\
                  .groupby('vaccines').count()['country'].reset_index().sort_values('country', ascending=False)

# Select only the required columns
vaccines.columns = ['Vaccines', 'Number of countries used']

fig = px.bar(vaccines, x='Vaccines', y= 'Number of countries used')
fig.show()